Genjax supports serializing traces into a byte format. To support different disk formats, GenJax offers different backends. Here we use `msgpack_serialize` which writes traces using the `MsgPack` protocol.

In [ ]:
import genjax
import jax
import jax.numpy as jnp
from genjax._src.core.serialization.msgpack import msgpack_serialize

In [ ]:
@genjax.static_gen_fn
def model(mu, coins):
    x = genjax.normal(mu, 1.0) @ "x"
    y = genjax.flip(jnp.sum(coins) / coins) @ "y"
    return x + y


key = jax.random.PRNGKey(314159)
tr = model.simulate(key, (-2.1, jnp.array([1, 1, 0])))
print(tr)

We can now convert `tr` into a byte representation using `serialize`. Note that GenJax provides `pickle`-like functions (e.g. `dumps`) that can alternatively be called instead. 

In [ ]:
payload = msgpack_serialize.serialize(tr)  # or msgpack_serialize.dumps(tr)

Deserialization is slightly different then one might be used to. In addition to passing in the bytes, the generative function is specified as well.

In [ ]:
retrieved_tr = msgpack_serialize.deserialize(payload, model)
print(retrieved_tr)

Note that serialized trace can be written to disk with the `MsgPack` backend. This means that even if the Python runtime terminates, the written trace can still be deserialized so long as the generative function induces the same trace structure. Observe that a fresh copy of `model` works just as well to deserialize the trace.

In [ ]:
@genjax.static_gen_fn
def model_copy(mu, coins):
    x = genjax.normal(mu, 1.0) @ "x"
    y = genjax.flip(jnp.sum(coins) / coins) @ "y"
    return x + y


retrieved_tr = msgpack_serialize.deserialize(payload, model_copy)
print(retrieved_tr)